In [ ]:
from google.colab import drive 
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
# Import all the necessary libraries
import re
import csv
import pandas as pd
import numpy as np 

In [ ]:
# Load data from csv file into a pandas dataframe
df_1 = pd.read_csv('gdrive/My Drive/Colab Notebooks/CA_youtube_trending_data.csv')
df_2 = df_1.iloc[120000:130000]
print(df_2)

In [ ]:
# Install and import the youtube transcript api
!pip install youtube_transcript_api
!pip install cs 
from youtube_transcript_api import YouTubeTranscriptApi

array = df_2.to_numpy() # Convert the pandas dataframe into a NumPy array

# Function to get transcript of a video
def generate_transcript(id):
	transcript = YouTubeTranscriptApi.get_transcript(id,languages=['en'])
	script = ""

	for text in transcript:
		t = text["text"]
		if t != '[Music]':
			script += t + " "
		
	return script, len(script.split())

# Loop through each video ID and get subtitles and write the data to a csv file
for i in range(len(array)):
  try:
    video_id=str(array[i][0])
    Channel_name = str(array[i][4])
    transcript, no_of_words = generate_transcript(video_id)
  except:
    transcript = ''
    Channel_name = ''
    print('The subtitles cannot be obtained')
  
  with open('gdrive/My Drive/Colab Notebooks/CAsubs13.csv', 'a',encoding='utf-8') as newFile:
    FileWriter = csv.writer(newFile)
    FileWriter.writerow([Channel_name, transcript])

In [ ]:
# Load all the data from from multiple csv files into pandas dataframes and join them into a single csv file 
data = pd.read_csv('gdrive/My Drive/Colab Notebooks/CAsubs.csv')
data_2 = pd.read_csv('gdrive/My Drive/Colab Notebooks/CAsubs2.csv')
data_3 = pd.read_csv('gdrive/My Drive/Colab Notebooks/CAsubs3.csv')
data_4 = pd.read_csv('gdrive/My Drive/Colab Notebooks/CAsubs4.csv')
data_final = pd.concat([data,data_2,data_3,data_4])
#data_final.rename(columns={0:'Name', 1:'Text'})
data_final.to_csv('gdrive/My Drive/Colab Notebooks/CAsubsnew.csv', encoding='utf-8')
print(data_final.shape)

(935, 2)


In [ ]:
# Drop rows with null values
df_3 = pd.read_csv('gdrive/My Drive/Colab Notebooks/CAsubsnew.csv')
df_4 = df_3.dropna(how='all')
df_4.to_csv('gdrive/My Drive/Colab Notebooks/CAsubs_2.csv', encoding='utf-8')
print(df_4.shape)

(475, 2)


In [ ]:
# Load data from csv file into a pandas dataframe and convert it into a NumPy array
df_5 = pd.read_csv('gdrive/My Drive/Colab Notebooks/CAsubs_2.csv')
array = df_5.to_numpy() 

# Loop through subtitles of each video and remove all special characters, unwanted spaces etc. and write it to a csv file
for k in range(len(array)):
  channel_name = str(array[k][0])
  text = str(array[k][1])
  new_text = re.sub(r"[^a-zA-Z]", " ", text)
  new = ""
  x = new_text.split()
  for i in range(len(x)):
    if len(x[i]) > 2:
      new = new + x[i] + " "
  
  with open('gdrive/My Drive/Colab Notebooks/CAsubs_3.csv', 'a',encoding='utf-8') as newFile:
    FileWriter = csv.writer(newFile)
    FileWriter.writerow([channel_name, new.lower()])

In [ ]:
# Sort the videos in alphabetical order
data_5 = pd.read_csv('gdrive/My Drive/Colab Notebooks/CAsubs_3.csv')
grouped_df = data_5.groupby("Name")
grouped_lists = grouped_df["Text"].agg(lambda column: "".join(column))
grouped_lists = grouped_lists.reset_index()
grouped_lists.to_csv('gdrive/My Drive/Colab Notebooks/CAsubsfinal.csv', encoding='utf-8')
print(grouped_lists)

In [ ]:
# Merge the data from subtitle files of different countries and sort them in alphabetical order 
data_6 = pd.read_csv('gdrive/My Drive/Colab Notebooks/CAsubsfinal.csv')
data_7 = pd.read_csv('gdrive/My Drive/Colab Notebooks/USsubsfinal.csv')
data_final_2 = pd.concat([data_6,data_7])
grouped_df_2 = data_final_2.groupby("Name")
grouped_lists_2 = grouped_df_2["Text"].agg(lambda column: "".join(column))
grouped_lists_2= grouped_lists_2.reset_index()
grouped_lists_2.to_csv('gdrive/My Drive/Colab Notebooks/US_CA_subs_all.csv', encoding='utf-8')
print(grouped_lists_2)

In [ ]:
print(len(grouped_lists_2['Name'].unique()))

566


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

df_plots = pd.read_csv('gdrive/My Drive/Colab Notebooks/US_CA_GB_subs_final.csv')

# Instantiate the vectorizer object to the vectorizer variable
vectorizer = TfidfVectorizer(max_df=0.7, min_df=2)

# Fit and transform the Text column
vectorized_data = vectorizer.fit_transform(df_plots['Text'].values.astype('U'))

# Look at the features generated
print(vectorizer.get_feature_names())

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Create Dataframe from TF-IDFarray
tfidf_df = pd.DataFrame(vectorized_data.toarray(), columns=vectorizer.get_feature_names())

# Assign the channel titles to the index and inspect
tfidf_df.index = df_plots['Name']
tfidf_df

,aaa,aaaaah,aaaah,aaah,aaahh,aah,aahed,aahhh,aaliyah,aang,...,zucker,zuckerberg,zuko,zulu,zuma,zunisha,zurich,zygarde,zyler,zyro
Name,,,,,,,,,,,,,,,,,,,,,
(G)I-DLE (여자)아이들 (Official YouTube Channel),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
100 Thieves,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
100% Star Wars,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
11Alive,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1320video,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
感谢订阅中剧独播,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
日本コロムビア 公式YouTubeチャンネル,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
스브스케이팝 / SBS KPOP,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
# Import cosine_similarity measure
from sklearn.metrics.pairwise import cosine_similarity

# Create the array of cosine similarity values
cosine_similarity_array = cosine_similarity(tfidf_df)

# Wrap the array in a pandas DataFrame
cosine_similarity_df = pd.DataFrame(cosine_similarity_array, index=tfidf_df.index, columns=tfidf_df.index)

# Print the top 5 rows of the DataFrame
cosine_similarity_df

Name,(G)I-DLE (여자)아이들 (Official YouTube Channel),100 Thieves,100% Star Wars,11Alive,1320video,15MOFERADIO,17 Duvy,1980sGamer,1theK (원더케이),1theK Originals - 원더케이 오리지널,...,zefrank1,Никита Ордынский,ポケモン公式YouTubeチャンネル,优优独播剧场——YoYo Television Series Exclusive,周杰倫 Jay Chou,感谢订阅中剧独播,日本コロムビア 公式YouTubeチャンネル,스브스케이팝 / SBS KPOP,스브스케이팝 X INKIGAYO,영국남자 Korean Englishman
Name,,,,,,,,,,,,,,,,,,,,,
(G)I-DLE (여자)아이들 (Official YouTube Channel),1.000000,0.012854,0.001626,0.001217,0.025387,0.002404,0.009047,0.009582,0.004067,0.004767,...,0.008823,0.009647,0.026135,0.019451,0.052105,0.018570,0.021936,0.024201,0.011925,0.024693
100 Thieves,0.012854,1.000000,0.010545,0.027164,0.223417,0.199476,0.077397,0.061477,0.019439,0.040685,...,0.045253,0.060515,0.036821,0.090794,0.014472,0.140565,0.048238,0.013792,0.029119,0.189106
100% Star Wars,0.001626,0.010545,1.000000,0.003034,0.015504,0.008997,0.015476,0.005678,0.002376,0.005213,...,0.007280,0.013572,0.002277,0.019937,0.002798,0.028977,0.004034,0.000000,0.000000,0.032143
11Alive,0.001217,0.027164,0.003034,1.000000,0.052567,0.006102,0.011716,0.003771,0.006004,0.004051,...,0.010992,0.008731,0.001462,0.026513,0.000673,0.019574,0.002456,0.000000,0.005810,0.020450
1320video,0.025387,0.223417,0.015504,0.052567,1.000000,0.167730,0.080082,0.046066,0.024918,0.075847,...,0.058465,0.047328,0.052376,0.092387,0.025650,0.127060,0.064108,0.084804,0.073796,0.231505
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
感谢订阅中剧独播,0.018570,0.140565,0.028977,0.019574,0.127060,0.090118,0.063999,0.040810,0.031954,0.066526,...,0.062751,0.101196,0.065597,0.212479,0.030848,1.000000,0.054224,0.014300,0.008145,0.149638
日本コロムビア 公式YouTubeチャンネル,0.021936,0.048238,0.004034,0.002456,0.064108,0.077208,0.105074,0.022944,0.019643,0.030461,...,0.017809,0.031084,0.072274,0.036440,0.016061,0.054224,1.000000,0.009613,0.000000,0.068673
스브스케이팝 / SBS KPOP,0.024201,0.013792,0.000000,0.000000,0.084804,0.000000,0.000000,0.000000,0.013554,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.014300,0.009613,1.000000,0.295838,0.006401


In [ ]:
# Find the values for the channel CNN
cosine_similarity_series = cosine_similarity_df.loc['CNN']

# Sort these values highest to lowest
ordered_similarities = cosine_similarity_series.sort_values(ascending=False)

# Print the results
ordered_similarities

Name
CNN                     1.000000
William Spaniel         0.290891
ABC News In-depth       0.177481
23 ABC News | KERO      0.177400
Vox                     0.154428
                          ...   
NCT                     0.000000
Mr. Montzingo           0.000000
MorrisonMade Leather    0.000000
Mork                    0.000000
Machine Gun Kelly       0.000000
Name: CNN, Length: 2817, dtype: float64

In [ ]:
list_of_channels_enjoyed = ['Marques Brownlee', 'iJustine',]

# Create a subset of only the channels the user has enjoyed
channels_enjoyed_df = tfidf_df.reindex(list_of_channels_enjoyed)

# Inspect the DataFrame
channels_enjoyed_df

,aaa,aaaaah,aaaah,aaah,aaahh,aah,aahed,aahhh,aaliyah,aang,...,zucker,zuckerberg,zuko,zulu,zuma,zunisha,zurich,zygarde,zyler,zyro
Name,,,,,,,,,,,,,,,,,,,,,
Marques Brownlee,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
iJustine,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
# Generate the user profile by finding the average scores of channels they enjoyed
user_prof = channels_enjoyed_df.mean()

# Inspect the results
print(user_prof)

aaa        0.0
aaaaah     0.0
aaaah      0.0
aaah       0.0
aaahh      0.0
          ... 
zunisha    0.0
zurich     0.0
zygarde    0.0
zyler      0.0
zyro       0.0
Length: 43182, dtype: float64


In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

# Find subset of tfidf_df that does not include channels in list_of_channels_enjoyed
tfidf_subset_df = tfidf_df.drop(list_of_channels_enjoyed, axis=0)

In [ ]:
# Calculate the cosine_similarity and wrap it in a dataframe
similarity_array = cosine_similarity(user_prof.values.reshape(1, -1), tfidf_subset_df)
similarity_df = pd.DataFrame(similarity_array.T, index=tfidf_subset_df.index, columns=["similarity_score"])

In [ ]:
# Sort the values from high to low by the values in the similarity_score
sorted_similarity_df = similarity_df.sort_values(by="similarity_score", ascending=False)

# Inspect the most similar to the user preferences
print(sorted_similarity_df.head())

                  similarity_score
Name                              
Apple                     0.512229
The Verge                 0.511247
Brian Tong                0.509425
The Everyday Dad          0.489007
Mac Address               0.480209
